# SIVmac239M analysis pipeline 
### Modified from Athena Golfinos, Ryan Moriarty and Brandon Keele
January 28, 2021

## Steps:
#### 1: Download files into the proper location 
#### 2: Demultiplex files 
#### 3: Barcoded virus analysis script from Brandon Keele
#### 4: Examination of barcode distribution using Simpson's Diversity index

## Importing Modules

In [ ]:
import os
import pandas as pd
import glob
from Bio import SeqIO
from Bio.Seq import Seq
import collections 
from fastqandfurious import fastqandfurious
from fastqandfurious.fastqandfurious import entryfunc
import regex
import matplotlib.pyplot as plt
import fnmatch
import matplotlib.backends.backend_pdf
import numpy as np
import csv
import itertools
import shutil
import scipy
import re
import sys
import seaborn as sb
from functools import reduce
import statistics




## Variables: User Input Necessary

ROOT = the absolute path to the fastq file that we need

EXPT = the name of the folder that we are looking at data within

INDICES = name of the file that contains all of our index tags. This file should be inside the EXPT folder which is then inside the ROOT directory. Either your file should be named "index_ids.csv", or you will need to edit the INDICES string below (that is the portion in red) 

BARCODES = this is the file that contains all of the reference barcode sequences for SIVmac239M. This should be contained within the ROOT directory, in a folder called "analysis", and the file should be named SIV239MReferenceSequences.fasta". If not, be sure to change the red test under "BARCODES" below. 



In [ ]:
ROOT = "/Users/agolfinos/Desktop/test"

EXPT = ""

INPUT = ROOT + EXPT + "/" + "rh2903-rh2909-rh2911-allsamples_S1_L001_R_001.fastq"

INDICES = pd.read_csv(ROOT + "/analysis/index_ids.csv", sep = ',', header = None)

#INDICES_5 = pd.read_csv(ROOT + "analysis/index_ids_5prime.csv", sep = ',', header = None)
idx5 = list(zip(INDICES[0], INDICES[1], INDICES[2]))

#INDICES_7 = pd.read_csv(ROOT + "analysis/index_ids_7prime.csv", sep = ",", header = None)
idx7 = list(zip(INDICES[0], INDICES[1], INDICES[2]))

BARCODES = [(b.id, b.seq) for b in SeqIO.parse(ROOT + "/analysis/SIV239MReferenceSequences.fasta", "fasta")]

pre_index = "GAGGTTCTGG"

pre_index_reverse_complement = "CCAGAACCTC"

post_index = "CTAGGGGAAG"

post_index_reverse_complement = "CTTCCCCTAG"

os.chdir(ROOT)

## Identifying the Orientation of the Reads and Reverse Transcribing Sequences, If Necessary

In [ ]:
def fastq_file_lists(INPUT):
    # pull in the sequences 
    seqs = []
    bufsize = 200000
    with open(INPUT, "rb") as fh:
        it = fastqandfurious.readfastq_iter(fh, bufsize, entryfunc)
        for sequence in it:
            # sequence[1] is the nucleotide sequence, sequence[2] is the quality scores 
            seqs.append((str(sequence[1])[2:-2], str(sequence[2])[2:-2]))

    preidx = []
    rcidx = []
    preidx_not_found = []
    for a in seqs:
        if pre_index in a[0]:
            preidx.append(a[0].index(pre_index))
        elif pre_index_reverse_complement in a[0]: 
            rcidx.append(a[0].index(pre_index_reverse_complement))
        else:
            preidx_not_found.append(a)
    seqs_rc = []
    if len(rcidx) > len(preidx):
        for a in seqs:
                seqs_rc.append((str(Seq(a[0]).reverse_complement()), a[1][::-1]))
        seqs = seqs_rc
        print("samples are reverse transcribed ")

    return seqs

## Finding all of the possible pre-index identifiers (Pre-Sequence GAGGTTCTGG)

In [ ]:
def pre_indexes(INPUT):
    
    #this calls the list of sequences that we created in the previous section
    all_seqs = fastq_file_lists(INPUT)
    
    #making an empty list to put our pre-index sequences in
    pre_index_permutations = []
    
    #adding the wild type sequence
    pre_index_permutations.append("GAGGTTCTGG")
    
    #now we are iterating through the list of all sequences
    for sequence in all_seqs: 
        
        #within the sequence, find the pre-index GAGGTTCTGG sequence, but allow for one mismatch to detect mutants
        m = regex.findall("(GAGGTTCTGG){s<=1}", str(sequence[0]))
        
        #now we are adding this sequence to our list of pre-index permutations
        pre_index_permutations.append(m)
    
    #what this is doing is flattening our 'list within lists', so making a single list that encompasses our sequences
    flat_list = [item for sublist in pre_index_permutations for item in sublist]
    
    #now, we are taking this list and ONLY keeping the UNIQUE values. This gets rid of all duplicates and provides us
    #with a list of just the unique identifiers we could find
    flat_list_set = set(flat_list)
    
    #this prevents any random sequences that aren't 10 nucleotides long from sneaking into our list by specifying that it must be 10nt long
    flat_list_set_int = [seq for seq in flat_list_set if len(seq) == 10]
    print("Our unique pre-index tag sequences are:")
    print(flat_list_set_int)
       
    return(flat_list_set_int)



## Finding all possible pre-index sequences (Pre-index sequence TCTCCAGCT)

In [ ]:
def pre_indexes(INPUT):
    
    #this calls the list of sequences that we created in the previous section
    all_seqs = fastq_file_lists(INPUT)
    
    #making an empty list to put our pre-index sequences in
    pre_index_permutations = []
    
    #adding the wild type sequence
    pre_index_permutations.append("TCTCCAGCT")
    
    #now we are iterating through the list of all sequences
    for sequence in all_seqs: 
        
        #within the sequence, find the pre-index GAGGTTCTGG sequence, but allow for one mismatch to detect mutants
        m = regex.findall("(TCTCCAGCT){s<=1}", str(sequence[0]))
        
        #now we are adding this sequence to our list of pre-index permutations
        pre_index_permutations.append(m)
    
    #what this is doing is flattening our 'list within lists', so making a single list that encompasses our sequences
    flat_list = [item for sublist in pre_index_permutations for item in sublist]
    
    #now, we are taking this list and ONLY keeping the UNIQUE values. This gets rid of all duplicates and provides us
    #with a list of just the unique identifiers we could find
    flat_list_set = set(flat_list)
    
    #this prevents any random sequences that aren't 10 nucleotides long from sneaking into our list by specifying that it must be 10nt long
    flat_list_set_int = [seq for seq in flat_list_set if len(seq) == 10]
    print("Our unique pre-index tag sequences are:")
    print(flat_list_set_int)
       
    return(flat_list_set_int)

## Single Index Demultiplex our FASTQ file using WT only pre-index sequence

Demultiplexes the Miseq file and separates them into individual samples
Args:
   file = the non-demultiplexed R1 or R2 file, save as .fastq
   indices = the P5.XX used to label each sample
        
Returns:
   list of demultiplexed samples:
   "Unknown_tag" - these are samples where the pre-index sequence "GAGGTTCTGG" or its reverese complement 
    was found, but the subseequent 8 base pairs, where the tag should be, was not present in the list of known 
    index tags. 
    "Unindexed" - these are samples where the pre-index sequence "GAGGTTCTGG" or its reverse complement was 
    NOT found, so we didn't look for the index tag either.
    All other samples will be labeled with their index tag "P5.XX"

In [ ]:
def demultiplex(INPUT, INDICES, write_files=True):
    
    file_end = INPUT[INPUT.rfind("/")+1:INPUT.rfind(".fastq")]
    all_seqs = fastq_file_lists(INPUT)
    
    index_names = list(INDICES[0])
    index_seqs = list(INDICES[1])
    index_seqs_rc = list(INDICES[2])
    
    # find where the indices are hiding 
    dmux_samples = dict()
    for a in all_seqs:
        if pre_index in a[0]:
            start = a[0].index(pre_index)+10
            tag = a[0][start:start+8]
            if tag in index_seqs:
                name = index_names[index_seqs.index(tag)] + "_rc"
                dmux_samples.setdefault(name, []).append(a)
            elif tag in index_seqs_rc:
                name = index_names[index_seqs_rc.index(tag)] 
                dmux_samples.setdefault(name, []).append(a)
            else:
                dmux_samples.setdefault("Unknown_tag", []).append(a)
        elif pre_index_reverse_complement in a[0]:
            start = a[0].index(pre_index_reverse_complement)+10
            tag = a[0][start:start+8]
            if tag in index_seqs:
                name = index_names[index_seqs.index(tag)] + "_rc"
                dmux_samples.setdefault(name, []).append(a) 
            elif tag in index_seqs_rc:
                name = index_names[index_seqs_rc.index(tag)] 
                dmux_samples.setdefault(name, []).append(a)
            else:
                dmux_samples.setdefault("Unknown_tag", []).append(a)
        else:
            dmux_samples.setdefault("Unindexed", []).append(a)

    index_keys = list(dmux_samples.keys())
    real_tags_found = len(list(filter(lambda element: 'P' in element, index_keys)))

    if real_tags_found > 0:
        if write_files:
            for i in index_keys:
                print("Writing sample index: ", i)
                data = dmux_samples.get(i)
                sample_fastq = open(file_end+ "_" + i + "_demultiplexed.fastq", "w")
                l = 0
                for line in data:
                    l += 1
                    sample_fastq.write("@" + i + "." + str(l) + "\n" + line[0] + "\n+\n" + line[1] + "\n" )
                sample_fastq.close()
    else:
        print("No known index tags found, try different read.")

    return dmux_samples

#print(fastqs[23])
demultiplexed = demultiplex(INPUT, INDICES)

## Single Index Demultiplex using a list of potential pre-index sequences

In [ ]:
def demultiplex(INPUT, INDICES, write_files=True):
    
    pre_index_list = pre_indexes(INPUT)
    file_end = INPUT[INPUT.rfind("/")+1:INPUT.rfind(".fastq")]
    all_seqs = fastq_file_lists(INPUT)
    
    index_names = list(INDICES[0])
    index_seqs = list(INDICES[1])
    index_seqs_rc = list(INDICES[2])
    
    # find where the indices are hiding 
    dmux_samples = dict()
    for a in all_seqs:
        count = 0
        for pre_index in pre_index_list:
            if pre_index in a[0]:
                start = a[0].index(pre_index)+10
                tag = a[0][start:start+8]
                if tag in index_seqs:
                    name = index_names[index_seqs.index(tag)] + "_rc"
                    dmux_samples.setdefault(name, []).append(a)
                elif tag in index_seqs_rc:
                    name = index_names[index_seqs_rc.index(tag)] 
                    dmux_samples.setdefault(name, []).append(a)
                count += 1
            elif pre_index_reverse_complement in a[0]:
                start = a[0].index(pre_index_reverse_complement)+10
                tag = a[0][start:start+8]
                if tag in index_seqs:
                    name = index_names[index_seqs.index(tag)] + "_rc"
                    dmux_samples.setdefault(name, []).append(a) 
                elif tag in index_seqs_rc:
                    name = index_names[index_seqs_rc.index(tag)] 
                    dmux_samples.setdefault(name, []).append(a)
                count +=1 
        if count == 0:
            dmux_samples.setdefault("Undetermined", []).append(a)

        index_keys = list(dmux_samples.keys())
        real_tags_found = len(list(filter(lambda element: 'P' in element, index_keys)))

    if real_tags_found > 0:
        if write_files:
            for i in index_keys:
                print("Writing sample index: ", i)
                data = dmux_samples.get(i)
                sample_fastq = open(file_end+ "_" + i + "_demultiplexed.fastq", "w")
                l = 0
                for line in data:
                    l += 1
                    sample_fastq.write("@" + i + "." + str(l) + "\n" + line[0] + "\n+\n" + line[1] + "\n" )
                sample_fastq.close()
    else:
        print("No known index tags found, try different read.")

    return dmux_samples

#print(fastqs[23])
demultiplexed = demultiplex(INPUT, INDICES)

## Dual Index Demultiplexing

This section takes the following as input: 

*F1 = this is the "R1" file from your run
*F2 = this is the "R2" file from your run
*index_keys_5 = this is the absolute path to a csv file that contains the names (column 1), sequences (column 2), and reverse complement of sequences (column 3) of your P5 index sequences from the primers. 
*index_keys_7 = this is the absolute path to a csv file that contains the names (column 1), sequences (column 2), and the reverse complement of sequences (column 3) of your P7 index sequences from the primers. 


## Dual Index Demultiplexing Variables: User Input Necessary

In [ ]:
F1 = ROOT + "/rh2903-rh2909-rh2911-allsamples_S1_L001_R1_001.fastq"
F2 = ROOT + "/rh2903-rh2909-rh2911-allsamples_S1_L001_R2_001.fastq"

index_keys_5 = pd.read_csv(ROOT + "/analysis/index_ids_5prime.csv", sep = ',', header = None)
idx5 = list(zip(index_keys_5[0], index_keys_5[1], index_keys_5[2]))

index_keys_7 = pd.read_csv(ROOT + "/analysis/index_ids_7prime.csv", sep = ",", header = None)
idx7 = list(zip(index_keys_7[0], index_keys_7[1], index_keys_7[2]))



In [ ]:
def fastq_file_info(F1, F2, write_files=True):
    
    file_end = F1[F1.rfind("/")+1:INPUT.rfind(".fastq")]
    
    #creating an empty list for our dual demultiplexed samples
    dmux_samples = dict()
    
    #pulling in our index names and sequences that we will use to demultiplex the R1
    index_names_R1 = list(index_keys_5[0])
    index_seqs_R1 = list(index_keys_5[1])
    index_seqs_rc_R1 = list(index_keys_5[2])
    
    #pulling in our index names and sequences that we will use to demultiplex the R2
    index_names_R2 = list(index_keys_7[0])
    index_seqs_R2 = list(index_keys_7[1])
    index_seqs_rc_R2 = list(index_keys_7[2])
    
    #pull in the sequences from F1
    headers1 = []
    seqs1 = []
    bufsize = 200000
    with open(F1, "rb") as fh:
        it = fastqandfurious.readfastq_iter(fh, bufsize, entryfunc)
        for sequence in it:
            #sequence[0] is the header, sequence[1] is the nucleotide sequence, sequence[2] is the quality scores
            headers1.append(str(sequence[0])[2:-9])
            seqs1.append(str(sequence[1])[2:-2] + "," + str(sequence[2])[2:-2])    
    #turning this list into a Pandas dataframe so we can more easily join the datasets from R1 and R2
    df1 = pd.DataFrame(headers1, columns=['Headers'])
    df1["Sequences R1"] = seqs1
            
    #pull in the sequences from F2
    headers2 = []
    seqs2 = []
    bufsize = 200000
    with open(F2, "rb") as fh: 
        it2 =fastqandfurious.readfastq_iter(fh, bufsize, entryfunc)
        for sequence in it2: 
            headers2.append(str(sequence[0])[2:-9])
            seqs2.append(str(sequence[1])[2:-2] + "," + str(sequence[2])[2:-2])

    #turning this list into a Pandas dataframe so we can more easily join the datasets from R1 and R2
    df2 = pd.DataFrame(headers2, columns=['Headers'])
    df2["Sequences R2"] = seqs2
    
    #joining dataframes 1 and 2 horizontally based on headers, combining all data from one read into one row
    df3 = pd.merge(df1, df2, on='Headers')
    
    #converting this dataframe to a list so we can better iterate through certain items in the list
    reads = df3.values.tolist()

####################END DATA ORGANIZATION AND CONCATENATION################################
####################NOW WE ARE ITERATING THROUGH OUR DATASET###############################
   
    #for one "sample"
    for item in reads: 
        
        #making a count of the number of real indexes found in this read--starting at zero
        count = 0 
        
        #if our pre_index sequence is in the R1 sequence: 
        if pre_index in item[1]: 
            #print(item[1])
            
            #find the 8bp tag
            start0 = item[1].index(pre_index)+8
            rtag0 = item[1][start0:start0+8]            
            
            #if this tag is in our list of tags
            if rtag0 in index_seqs_R1: 
            
                #we have found a "real" tag, so we will add to the count for this read
                count+=1
                
                #then, look for the post-index tag in the forward read (R2)
                if post_index in item[2]:
                    #print(item[2])
                    
                    #now we want to find our index tag relative to our post-index sequence
                    start5 = item[2].index(post_index)-8
                    tag5 = item[2][start5:start5+8]
                    
                    #if our identified P7 index tag is in the list of pre-defined P7 index tags
                    if tag5 in index_seqs_R2: 
                        
                        count+=1
                        
                        #we are giving our dual indexed read a name--the P5 and P7 index tags
                        name = index_names_R1[index_seqs_rc_R1.index(tag1)] + "_" + index_names_R2[index_seqs_R2.index(tag2)]
                        dmux_samples.setdefault(name, []).append(item[1])
                elif post_index_reverse_complement in item[2]: 
                    print("RC post index in this sample")
                        
        #if our pre index reverse complement sequence is in the R1 sequence
        elif pre_index_reverse_complement in item[1]:
            #print("Our R1 read is:" + item[1])
            
            #find the 8bp tag in the R1 read
            start1 = item[1].index(pre_index_reverse_complement)-8
            tag1 = item[1][start1:start1+8]
            
            #if this tag is in our list of R1/P5 tags
            if tag1 in index_seqs_rc_R1:
                
                #we have found a "real" P5 tag, so we will add to the count for this read
                count+=1
                
                #then we want to see if our "post index" is in the R2 read
                if post_index in item[2]: 
                    
                    #now we want to find our index tag relative to our index sequence
                    start2 = item[2].index(post_index)-8
                    tag2 = item[2][start2:start2+8]
                    
                    #if our identified P7 index tag is in the list of pre-defined P7 index tags
                    if tag2 in index_seqs_R2: 
                        
                        #we have found a "real" tag, so we will add to the count for this read
                        count+=1
                        
                        #we are giving our dual indexed read a name--the P5 and P7 indexes combined
                        name = index_names_R1[index_seqs_rc_R1.index(tag1)] + "_" + index_names_R2[index_seqs_R2.index(tag2)] 
                        dmux_samples.setdefault(name, []).append(item[1])
                        
                elif post_index_reverse_complement in item[2]: 
                    print("RC post index in this RC sample")

        if count == 0:
            dmux_samples.setdefault("Undetermined", []).append(item[1])
        
        
    
###################GETTING A LIST OF INDEX KEYS AND THEIR COUNTS###########################
    
    #this provides a list of index keys (samples) as well as their corresponding read counts
    for k,v in dmux_samples.items():
        print (k, len(list(filter(None, v))))

    #compliling a list of all our index keys, or samples to be demultiplexed
    index_keys = list(dmux_samples.keys())
    
    #this counts the list of samples to be demultiplexed
    real_tags_found = len(list(filter(lambda element: 'P' in element, index_keys)))

##################IF WE HAVE REAL DATA, WE WILL NOW COMPILE IT AND WRITE TO FILES##########
    
    #if our list of real tags is greater than 0
    if real_tags_found > 0:
        if write_files:
            
            #for each of the index keys
            for i in index_keys:
                print("Writing sample index: ", i)
                
                #this is the sequence/qual scores of all the reads that fell under this index key
                data = dmux_samples.get(i)
                
                #opening the fastq file for this particular index 
                sample_fastq = open(file_end + "_" + i + "_demultiplexed.fastq", "w")

                l = 0
                
                #for each read + qual score entry in the data set for this index
                for entry in data: 
                    print(entry)
                    l += 1
                
                #this splits the sequence/qual score string into two parts 
                #sequence is lines[0] and qual score is lines [1]
                    lines = entry.split(",")               
                    sample_fastq.write("@" + i + "." + str(l) + "\n" + lines[0] + "\n+\n" + lines[1] + "\n" )
                sample_fastq.close()
    else:
        print("No known index tags found, try different read.")

    return dmux_samples 

fastq = fastq_file_info(F1, F2)

---------------------------------------------------------------------------------------------
# Barcode Analysis (not working)

In [ ]:
def find_barcodes_v2(file_dict, barcode_file):
    """ 
    Finds the barcodes present in the demultiplexed files 
    
    Args:
        files: list of demultiplexed files as a SeqIO-parsed generator
        barcode_file: list of barcodes to search for in the samples, as a SeqIO-parsed generator 
        
    Returns:
        a collections.Counter object for each sample, describing {barcode: counts}
        
    """
    
    
    for f in file_dict:
        if "P" in f:
            print("Starting barcode search for ", f)
            sample = file_dict.get(f)
            sequence = list(zip(*sample))[0]
            barcodes_found = []
            for b in BARCODES:
                if b[1] in sequence:
                    barcodes_found.append(b[0])
            print(f, collections.Counter(barcodes_found))
        
        #for sample in sample_indices:
        #    print(sample, len(sample_seqs[sample]))
        #    sequences = list(zip(*sample_seqs[sample]))
        #    barcodes_found = []
        #    for b in BARCODES:
        #        print(b[0])
        #        if b[1] in sequences:
        #            barcodes_found.append(b[0])
        #    print(collections.Counter(barcodes_found))
       
    
    #return all_barcodes 

find_barcodes_v2(demultiplexed, BARCODES)

In [ ]:
def find_barcodes(files, barcode_file):
    """ 
    Finds the barcodes present in the demultiplexed files 
    
    Args:
        files: list of demultiplexed files as a SeqIO-parsed generator
        barcode_file: list of barcodes to search for in the samples, as a SeqIO-parsed generator 
        
    Returns:
        a collections.Counter object for each sample, describing {barcode: counts}
        
    """
    
    all_barcodes = []
    i = 0
    for f in files:
        i += 1
        print("Starting sample ", i) #so we are sure we are still progressing 
        barcodes_found = []
        for sample in f:
            for b in barcode_file:
                if b[1] in sample.seq:
                    barcodes_found.append(b[0])
        barcode_counts = collections.Counter(barcodes_found)
        all_barcodes.append(barcode_counts)
    return all_barcodes 

In [ ]:
def barcode_counts(barcode_list):
    """ 
    Take the collections.Counter objects generated by find_barcodes 
    and write a csv of the barcodes identified in that sample 
    
    Args:
        barcode_list: the list of collections.Counter objects 
    
    Returns:
        a csv written into the working directory of the barcodes identified and their count 
    
    """
    
    for sample in barcode_list:
        if len(sample) > 0:
            
            total_counts = sum(sample.values())
            num_barcodes = len(sample)
            sampledict = dict(sample)
            print(list(sampledict.keys()), sampledict.values())
            #for s in range(0, num_barcodes):
            #    print(dict(sample)[s])
            print("number of unique barcodes:", num_barcodes)
            print(sample.keys())
        
    return total_counts, num_barcodes

barcode_counts(bcs_found)

--------------------------------------------------------------------------------------------
## Simpson's Diversity Index

## Defining Global Variables: User Input Necessary

In [ ]:
%matplotlib inline

######INFORMATION ABOUT THE DATA YOU ARE INPUTTING########

#what is the absolute path to the directory that contains all of your data? 
appRoot = "/Users/agolfinos/Desktop/Prophylactic_Vaccines_Barcode_Data/rh2909_rh2911_rh2921"

#FOLDER NAMES

rawBarcodes = appRoot + "/3.BarcodeData/"

splittingBarcodeAnalysisOutputDirectory = appRoot + "/4.SplitBarcodeFiles"

summarySpreadsheetDirectory = appRoot + "/5.AllDataToBeUsed"

sdiDirectory = appRoot + "/6.CalculatingSDI"

sdiFinalSpreadsheets = appRoot + "/7.SDI"

#mhFolder = appRoot + "/8.Morisita-Horn"


#REPLICATES FOR THE REFERENCE AND EXPERIMENTAL SAMPLING STEPS 

replicates = 10


#STATISTICS SPREADSHEETS

animal_1_folders = glob.glob(summarySpreadsheetDirectory + "/BarcodeList/" + "*")


#MORISITA-HORN FILES

#animal1MH = mhFolder + "/MHmatrix" + animal1name + ".csv"


## Spreadsheet Handling and File Manipulation: Creating Necessary Directories

This portion of the code makes all of the necessary directories for you to properly run this code. 

In [ ]:
def making_directories():    
    if not os.path.exists(splittingBarcodeAnalysisOutputDirectory):
        os.makedirs(splittingBarcodeAnalysisOutputDirectory)
    
    if not os.path.exists(summarySpreadsheetDirectory): 
        os.makedirs(summarySpreadsheetDirectory)
        
    if not os.path.exists(sdiDirectory): 
        os.makedirs(sdiDirectory)
        
    if not os.path.exists(sdiFinalSpreadsheets): 
        os.makedirs(sdiFinalSpreadsheets)
        
    #if not os.path.exists(mhFolder): 
        #os.makedirs(mhFolder)
        
directories = making_directories()

## Copying Files To Splitting Barcodes Directory

This portion of the script will copy everything from the barcode analysis directory and copy it into the splitting barcodes directory so we can run the next portion of the script, which splits these files into two parts. We want to copy these files into a new directory instead of using the old directory because we want to save the original files. 

In [ ]:
def moving_barcode_analysis_files(): 
    
    barcodes = os.listdir(rawBarcodes)
    destpath = splittingBarcodeAnalysisOutputDirectory
    for f in barcodes:
        shutil.copy(rawBarcodes + "/" + f, destpath)
            
moving_files = moving_barcode_analysis_files()

## Splitting Barcode Analysis Output Files

This is the first step in the post-barcode analysis pipeline. It will take the output files and split them into two parts. This is because the original file has some "summary stats" at the top with certain column headers, and the second half of the sheet has different information and different column headers. Since this can be kind of a pain when we want to analyze this going forward, we split these files into two parts to prepare for the later analysis.

In [ ]:
#call all of the csv files in the cwd and makes a list of all of them
def listfilenames(): #this writes all file names to a list
    results = []
    for root, dirs, files in os.walk(splittingBarcodeAnalysisOutputDirectory):
        for filename in files:
            if filename.endswith('.csv'): #this will be where we store the names of our files
                results.append(filename)
    return(results)

def move_rows(results):
    for fname in results:
        file_in = splittingBarcodeAnalysisOutputDirectory + "/" + fname
        file_out = splittingBarcodeAnalysisOutputDirectory + "/" + "SUMMARY_"  + fname
        file_temp = splittingBarcodeAnalysisOutputDirectory + "/" + "TEMP_" + fname
        with open(file_in, "r") as f_input, open(file_out, "a") as f_output, open(file_temp, "w") as f_temp:
            csv_input = csv.reader(f_input)
            
            #append first 4 rows to file_out
            csv.writer(f_output).writerows(itertools.islice(csv_input, 0, 4))
            
            #write the remaining rows from file_in to file_temp
            csv.writer(f_temp).writerows(csv_input)
        os.remove(file_in)
        os.rename(file_temp, splittingBarcodeAnalysisOutputDirectory + "/" "REVISED_STATS_" + fname)

            
results = listfilenames()
moving_rows = move_rows(results)

## Revised Stats Spreadsheet Manipulation

This step will first move all of the revised stats files into the summary spreadsheet directory. From there, we use the file names from each of the revised stats files (edited so we are only using a portion of the original name) to create directories named after each of the files. Then, after it creates a directory named after each of the files, it moves the file into the folder of the matching name. This is so that each file has its own directory to manage all of the summary sheets that are created in the next step. 

In [ ]:
#moving these files to a new folder 
def relocate_revised_stats():
    source = splittingBarcodeAnalysisOutputDirectory
    dest1 = summarySpreadsheetDirectory
    files = os.listdir(source)
    for f in files: 
        if f.startswith("REVISED_STATS"):
            full_file_name = os.path.join(source, f)
            if (os.path.isfile(full_file_name)):
                shutil.copy(full_file_name, dest1)

#makes a list of all directories so we can use this to make directories 
def make_directories_list():
    for root, dirs, files in os.walk(summarySpreadsheetDirectory):
        for filename in files:
            if filename.endswith('.csv'): #this will be where we store the names of our files
                
                #########ALTER YOUR NAME SELECTION CRITERIA HERE##############
                f = re.split('_', filename)
                f1 = f[2:5] #these only keep the part of the name that you need and omits the other parts of the name
                f2 = '_'.join(f1)
                ###############################################################
                
                if not os.path.exists(summarySpreadsheetDirectory + "/" + f2):
                    os.makedirs(summarySpreadsheetDirectory + "/" + f2) #MAKES DIRECTORY FOR THE FILE
                shutil.move(summarySpreadsheetDirectory + "/" + filename, summarySpreadsheetDirectory + "/" + f2) #MOVES THE NAME MATCHED FILE INTO ITS CORRESPONDING DIRECTORY 

relocating_revised_stats = relocate_revised_stats()
making_directories_list = make_directories_list()

## Creating Summary Spreadsheets 1-3

This is the part of the code that will take the split and relocated files from the barcode virus analysis tool and will turn them into 3 separate spreadsheets that we will use as part of the rest of this analysis. 

In [ ]:
#prevent the setting with copy warning by disabling chained assignment
pd.options.mode.chained_assignment = None

def identify_barcodes(f2):
    
    #creates an empty pandas dataframe to read the results to
    df = pd.DataFrame([])
    
    #looks only for files that start with REVISED_STATS and ends with csv
    for counter, file in enumerate(glob.glob("REVISED_STATS*.csv")):
        
        #this only uses the first and third columns of the csv file, sets the index column as the barcode name
        namedf = pd.read_csv(file, usecols=[0,2])
        df = df.append(namedf)
        
        if file.endswith('.csv'): #this will be where we store the names of our files
            
            #############ALTER YOUR NAMING SCHEME HERE####################
            f = file[22:]
            f1 = f[:-37] #these only keep the part of the name that you need and omits the other parts of the name
            f2 = str(f1)
            ##############ALTER YOUR NAMING SCHEME HERE##################
        
    #filters out unique barcodes and only keeps the SIV barcodes
    df1 = df[df["Barcode"].str.contains("SIV", na=False)]
    
    #here we sum up the total barcode count
    total_count = df1["Counts"].sum()
    
    #here we add the column with the total barcode count
    df1["Total_Barcode_Count"] = total_count
    
    #here we create the frequency column
    df1['Percent_Composition'] = df1['Counts']/df1['Total_Barcode_Count']
    
    #creates a data frame with the raw counts of all the barcodes found in at least 0.1% proportion
    df2 = df1[["Barcode", "Counts"]]
    
    #prints the filtered barcode count data to a csv
    df2.to_csv("1_" + str(f2) + "_Filtered_Barcode_Counts.csv", index=False)
    
    #read in the source file to a pandas dataframe
    fname = "1_" + str(f2) + "_Filtered_Barcode_Counts.csv"
    df = pd.read_csv(fname)
    
    #here we sum up the total barcode count
    total_count = df["Counts"].sum()
    
    #here we add the column with the total barcode count
    df["Total_Barcode_Count"] = total_count
    
    #here we create the frequency column
    df['Percent_Composition'] = df['Counts']/df['Total_Barcode_Count']
    
    #making a new column that includes the path name 
    idx = 0
    new_col = path
    df.insert(loc=idx, column="Sample_Name", value=new_col)
    
    #making a dataframe that only has proportion so we are able to plot this
    df1 = df[["Barcode","Percent_Composition"]]
    
    #printing the dataframes to their own spreadsheets
    df.to_csv("2_" + path + "_Filtered_Barcode_Counts_and_Percentage.csv", index=False)
    df1.to_csv("3_" + path + "_Filtered_Barcode_Percentage.csv", index=False)
    
#executing workflow
#this automates the running of the code by iterating through the directories
os.chdir(summarySpreadsheetDirectory)
for i in os.listdir(summarySpreadsheetDirectory):
    
    #ignores hidden directory .DS_Store
    if i == ".DS_Store":
        continue
        
    print(i)
    
    #changes directory to the next one in the list
    os.chdir(i)
    
    #changes the path variable to the directory you just moved into
    path = i
    
    #print an update statement
    print("Starting analysis for the " + path + " directory")
    
    #execution of the functions
    listing_barcodes = identify_barcodes(making_directories_list)
    
    #printing a update statement
    print("Finished analysis for the " + path + " directory\n\n")
    
    #moves the directory back up to the original directory
    os.chdir("../")
    

## Simpson's Diversity Index Calculation

In [ ]:
############DEFINING THE FUNCTION#####################################################################################

def SDI(): 
    
###########NOW OPENING THE FIRST FILE TO EXTRACT THE BARCODES AND ADD THEM TO THE TOTAL POOL##########################    
    
    SDI_file_names = []
    
    SDImedians = []
    
    for roots, dirs, files in os.walk(summarySpreadsheetDirectory): 
            
        for dir in dirs: 
            
            all_barcodes1 = []
        
            #splitting the path name so we can make a list of all the samples as we iterate through the files
            split_i = re.split('/', summarySpreadsheetDirectory + "/" + dir)
            sampleName = split_i[-1]

            #adding the file name to the list of filenames to use later for an index column and row
            SDI_file_names.append(sampleName)

            #creates a list of all of the files in this directory
            directory = os.listdir(summarySpreadsheetDirectory + "/" + dir)

            #filtering the list for only the second spreadsheet, which is the only one we will be using
            first_file = [s for s in directory if s.startswith("1_")]

            #converting the file name to a string
            string_first_file = ''.join(first_file)
            
            path = str(summarySpreadsheetDirectory + "/" + dir)

            #gives us the absolute full path to the first spreadsheet in that directory
            string_path_1 = os.path.join(path, string_first_file)

            #converts this file to a pandas dataframe
            df1 = pd.read_csv(string_path_1, header=[0])
            
            #iterates through rows of the pandas dataframe
            for index, row in df1.iterrows(): 

                #creates a "counts" value that gives the number of times that this barcode is found in that file
                counts1 = row['Counts']
                counts1 = int(counts1)

                #gives us the name of the barcode associated with the count right above
                barcode1 = row['Barcode']

                #this appends all the barcode names to the list in the range that is given in the counts column
                all_barcodes1.extend([barcode1 for x in range(counts1)])

            #convert all_barcodes to a numpy array 
            array1 = np.array(all_barcodes1)

###########NOW SETTING THE LOOP THAT WILL RUN THIS PIPELINE IN REPLICATE##############################################
            
            #this is going to create a list of all SDI values creates during this step so we can find the median
            #value and make that the final value 
            SDIreplicates = []
        
            #this is going to run the "picking" from the all_barcodes array a fixed number of times ("replicates" value)
            for x in range(0, replicates):
                
###########NOW RUNNING SDI CALCULATION ON THIS DATAFRAME##############################################################
                
                #this only uses the counts column 
                #f2 = f1['Counts']
                f2 = df1['Counts']
                
                #this turns the column into a numpy array to convert them into integers
                a = np.array(f2)
                
                #sums the counts column (should match the number of reads we pulled out)
                N = df1['Counts'].sum()
                
                #creates a list to turn into the number for the numerator
                numerator = 0
                
                #creates the sum to be used in the numerator
                for i in a: 
                    numerator = numerator + i*(i-1)
                
                #this actually calculates the Simpson diversity index
                SDI = float(1-(numerator/(N*(N-1))))
                
                #this appends the SDI value to the list
                SDIreplicates.append(SDI)
            
            #this finds the median value from the list of replicates 
            SDImedian = statistics.median(SDIreplicates)
            
            #adding this value to the list of SDI medians
            SDImedians.append(SDImedian)
            
            #sets the new file name equal to a variable so it can be created later 
            fname = sdiDirectory + "/" + 'Simpson_Diversity_Index_allreads_' + str(replicates) + "replicates.csv"
            
    #zips the SDI file names to the corresponding median values to use to make a spreadsheet
    SDI_values = dict(zip(SDI_file_names, SDImedians))

    #converts this first dictionary to a pandas dataframe so we can save it as a csv file
    SDI_df = pd.DataFrame(SDI_values.items(), columns=['Time_Point', 'SDI_Value'])
    
    #save this dataframe to a csv 
    SDI_df.to_csv(fname, columns=['Time_Point', 'SDI_Value'])
    
    SDI_df.to_csv(sdiFinalSpreadsheets + "/" + "SDI_allreads_" + str(replicates) + "reps_.csv", index=False)
    
    
Simpsons = SDI()